I need a Data set like this:

In [63]:
'''
   Yield  Property1  Property2  ParentMaterial1  ParentMaterial2
0     10          3          5                2                4
1     12          2          6                3                5
2     15          4          7                2                4
3     11          2          5                3                5
4     13          3          6                2                4
'''

'\n   Yield  Property1  Property2  ParentMaterial1  ParentMaterial2\n0     10          3          5                2                4\n1     12          2          6                3                5\n2     15          4          7                2                4\n3     11          2          5                3                5\n4     13          3          6                2                4\n'

Property 1 and Property2 will be features of output batches. ParentMaterial1 and ParentMaterial2 will be features of input batches.

In [64]:
import pandas as pd
import numpy as np

#data folder path definition
data_path='c:\\Users\\seyma\\personal_work\\sanofi\\Sanofi_Challenge\\Data\\'

#first load the excel document
Excel_file=pd.ExcelFile(data_path+"Datainput.xlsx")

#create data frames for each sheet in the excel file
Combined_Yields=pd.read_excel(Excel_file,sheet_name="Combined_Yields")
QC_Data=pd.read_excel(Excel_file,sheet_name="QC_Data")
IoT=pd.read_excel(Excel_file,sheet_name="IoT")
Campaign_Info=pd.read_excel(Excel_file,sheet_name="Campaign_Info")
Batch_Genealogy=pd.read_excel(Excel_file,sheet_name="Batch_Genealogy")

1- Create new_id Column, fix date format

In [65]:
# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in QC_Data
QC_Data['new_id'] = QC_Data['BATCH_ID'].astype(str) + QC_Data['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in Combined_Yields_full
Combined_Yields['new_id'] = Combined_Yields['BATCH_ID'].astype(str) + Combined_Yields['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in Batch_Genealogy
Batch_Genealogy['new_id'] = Batch_Genealogy['BATCH_ID'].astype(str) + Batch_Genealogy['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in Campaign_Info
Campaign_Info['new_id'] = Campaign_Info['BATCH_ID'].astype(str) + Campaign_Info['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in IoT
IoT['new_id'] = IoT['BATCH_ID'].astype(str) + IoT['MATERIAL_ID']


QC_Data['MANUFACTURE_DATE'] = QC_Data['MANUFACTURE_DATE'].replace(0, np.nan)
Combined_Yields['MANUFACTURE_DATE'] = Combined_Yields['MANUFACTURE_DATE'].replace(0, np.nan)
Batch_Genealogy['MANUFACTURE_DATE'] = Batch_Genealogy['MANUFACTURE_DATE'].replace(0, np.nan)

Combined_Yields['MANUFACTURE_DATE'] = pd.to_datetime(Combined_Yields['MANUFACTURE_DATE'], format='%Y%m%d')
QC_Data['MANUFACTURE_DATE'] = pd.to_datetime(QC_Data['MANUFACTURE_DATE'], format='%Y%m%d')
Batch_Genealogy['MANUFACTURE_DATE'] = pd.to_datetime(Batch_Genealogy['MANUFACTURE_DATE'], format='%Y%m%d')



2- Drop Unneccesary Colmuns and Rows and Duplicates

In [66]:
#drop na values in date column
Combined_Yields.dropna(subset=['MANUFACTURE_DATE'], inplace=True)
QC_Data.dropna(subset=['MANUFACTURE_DATE'], inplace=True)
Batch_Genealogy.dropna(subset=['MANUFACTURE_DATE'], inplace=True)

Combined_Yields_full=Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"].dropna()
Combined_Yields_full=Combined_Yields_full.drop(['BATCH_ID','MATERIAL_ID','STAGE','YIELD_NAME'], axis=1)

# Delete rows where MOVEMENT_TYPE is equal to 531 and Remove the LEVEL column
Batch_Genealogy = Batch_Genealogy[Batch_Genealogy['MOVEMENT_TYPE'] != 531]
Batch_Genealogy = Batch_Genealogy.drop(['BATCH_ID','MATERIAL_ID','LEVEL','TREE_ID','NEW_MATERIAL_DESCRIPTION'], axis=1)

Batch_Genealogy = Batch_Genealogy.drop_duplicates()

QC_Data = QC_Data.drop(['BATCH_ID','MATERIAL_ID'], axis=1)
Campaign_Info = Campaign_Info.drop(['BATCH_ID','MATERIAL_ID'], axis=1)
IoT = IoT.drop(['BATCH_ID','MATERIAL_ID'], axis=1)


3- Add output_id on Batch_Genealogy data

In [67]:
Batch_Genealogy

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION,new_id
0,R104490026,2022-02-03,232.100,KG,101,hep_so,122812P10449
1,R104490027,2022-02-11,253.550,KG,101,hep_so,122813P10449
2,R104490036,2022-09-16,265.100,KG,101,hep_so,122825P10449
3,R104490023,2021-11-05,223.900,KG,101,hep_so,121843P10449
4,R104500007,2021-10-21,227.200,KG,101,hep_so,121823P10450
...,...,...,...,...,...,...,...
4187,R104500005,2020-12-29,26.347,KG,261,hep_bru_dc,121905P10010
4189,R104500005,2021-01-05,22.910,KG,261,hep_bru_dc,121906P10010
4191,R104500005,2021-01-06,25.035,KG,261,hep_bru_dc,121907P10010
4193,R104500005,2021-01-13,26.041,KG,261,hep_bru_dc,121908P10010


In [68]:
# Function to get new_id for MOVEMENT_TYPE=101 based on PROCESS_ORDER_NUMBER
def get_output_id(row):
    if row['MOVEMENT_TYPE'] == 261:
        same_order_rows = Batch_Genealogy[(Batch_Genealogy['PROCESS_ORDER_NUMBER'] == row['PROCESS_ORDER_NUMBER']) & (Batch_Genealogy['MOVEMENT_TYPE'] == 101)]
        if len(same_order_rows) > 0:
            return same_order_rows.iloc[0]['new_id']
    elif row['MOVEMENT_TYPE'] == 101:
        return 0
    
    return None

# Apply the function to create the 'output_id' column
Batch_Genealogy['output_id'] = Batch_Genealogy.apply(get_output_id, axis=1)


In [69]:
#Batch_Genealogy.to_excel("C:\\Users\\seyma\\Downloads\\batch.xlsx")

In [74]:
Batch_Genealogy

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION,new_id,output_id
0,R104490026,2022-02-03,232.100,KG,101,hep_so,122812P10449,0
1,R104490027,2022-02-11,253.550,KG,101,hep_so,122813P10449,0
2,R104490036,2022-09-16,265.100,KG,101,hep_so,122825P10449,0
3,R104490023,2021-11-05,223.900,KG,101,hep_so,121843P10449,0
4,R104500007,2021-10-21,227.200,KG,101,hep_so,121823P10450,0
...,...,...,...,...,...,...,...,...
4187,R104500005,2020-12-29,26.347,KG,261,hep_bru_dc,121905P10010,121821P10450
4189,R104500005,2021-01-05,22.910,KG,261,hep_bru_dc,121906P10010,121821P10450
4191,R104500005,2021-01-06,25.035,KG,261,hep_bru_dc,121907P10010,121821P10450
4193,R104500005,2021-01-13,26.041,KG,261,hep_bru_dc,121908P10010,121821P10450


Create a Data Frame for Input Materials

In [79]:
output_new_id_list=Batch_Genealogy[Batch_Genealogy["MOVEMENT_TYPE"]==101]["new_id"].unique()
output_materials=Combined_Yields_full["new_id"].unique() #108 materials
common_ids = set(output_materials).intersection(set(output_new_id_list))
print(common_ids)
print("Length of intersection: {}".format(len(common_ids)))

{'122815P10449', '122813P10449', '122802P10450', '120936P10449', '122823P10449', '121832P10444', '121828P10449', '121843P10444', '121824P10450', '121820P10450', '122827P10449', '120928P10449', '122795P10450', '120928P10446', '121844P10449', '120941P10444', '122804P10450', '120936P10444', '123711P10444', '122817P10449', '121831P10444', '120938P10449', '120934P10446', '122826P10449', '121835P10449', '120934P10449', '121834P10449', '120930P10444', '120945P10444', '121826P10449', '120935P10444', '122803P10450', '121837P10449', '122821P10449', '122799P10450', '121845P10449', '120932P10446', '121836P10444', '122796P10450', '120937P10449', '121822P10450', '120943P10444', '121834P10444', '121832P10449', '122825P10449', '121840P10444', '120932P10444', '122812P10444', '121827P10449', '121838P10444', '122824P10449', '121842P10449', '121818P10450', '122788P10443', '121833P10444', '120928P10444', '120937P10444', '120944P10444', '122801P10450', '122814P10444', '121816P10450', '121839P10449', '122829

In [103]:
Batch_Genealogy_input = Batch_Genealogy[Batch_Genealogy['output_id'].isin(common_ids)].copy()

#saving input ids
input_id_list=Batch_Genealogy_input.new_id.unique()

Batch_Genealogy_input

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION,new_id,output_id
282,R104440013,2020-04-27,2.000,1,261,fil22,113513P20098,120944P10444
283,R104440013,2020-05-05,4.000,1,261,fil45,113526P20097,120944P10444
288,R104440024,2020-06-09,24.000,KG,261,potasi,113575P20003,121844P10444
290,R104440019,2020-06-09,24.000,KG,261,potasi,113575P20003,121836P10444
292,R104440011,2020-04-27,2.000,1,261,fil22,113513P20098,120942P10444
...,...,...,...,...,...,...,...,...
4187,R104500005,2020-12-29,26.347,KG,261,hep_bru_dc,121905P10010,121821P10450
4189,R104500005,2021-01-05,22.910,KG,261,hep_bru_dc,121906P10010,121821P10450
4191,R104500005,2021-01-06,25.035,KG,261,hep_bru_dc,121907P10010,121821P10450
4193,R104500005,2021-01-13,26.041,KG,261,hep_bru_dc,121908P10010,121821P10450


In [101]:
tree = Batch_Genealogy_input.groupby('output_id')['new_id'].unique()
tree2 = Batch_Genealogy_input.groupby('output_id')['new_id'].nunique()
tree=pd.merge(tree,tree2,on="output_id")
tree=tree.rename(columns={"new_id_x":"input id list","new_id_y":"# of inputs"})

In [102]:
tree

,input id list,# of inputs
output_id,,
120928P10444,"[115685P10004, 115693P10004, 117299P10004, 112...",12
120928P10446,"[112772P20098, 112891P20093, 112892P20095, 112...",12
120928P10449,"[112891P20093, 112892P20095, 112923P20097, 112...",30
120930P10444,"[117299P10004, 112892P20095, 112959P20094, 113...",11
120930P10446,"[112891P20093, 112892P20095, 112923P20097, 112...",12
...,...,...
122826P10449,"[114256P20095, 114277P20003, 114426P20094, 114...",21
122827P10449,"[114256P20095, 114426P20094, 114702P20002, 114...",21
122828P10449,"[114256P20095, 114426P20094, 114721P20093, 114...",21


In [98]:
#tree.to_excel("C:\\Users\\seyma\\Downloads\\output_3.xlsx")

Filtering QC results for input materials

In [111]:
QC_Data_input = QC_Data[QC_Data['new_id'].isin(input_id_list)].copy()
QC_Data_input

,MANUFACTURE_DATE,NEW_QC_TEST,VALUE,new_id
3,2020-03-18,PH,6.9,120932P10010
23,2021-05-19,perte_cre1,NaN,121942P10010
26,2022-05-23,perte_cre1,NaN,123544P10005
34,2021-10-21,lod,6.0,122888P10010
49,2022-06-18,pere_pour_azo,NaN,123274P10004
...,...,...,...,...
428898,2021-11-10,pere_pour_azo,NaN,122895P10010
428903,2021-08-23,cend_sul,NaN,122595P10005
428925,2021-10-13,lla_tel,NaN,121962P10010
428928,2022-06-14,xa_tel,NaN,122973P10010


In [116]:
value_counts = QC_Data_input["new_id"].value_counts().sort_index()

# Display the counts
print(value_counts)

115685P10004    74
115693P10004    74
117268P10004    74
117299P10004    74
119103P10009    74
                ..
123613P10005    74
123614P10005    74
123615P10005    74
123616P10005    74
123617P10005    74
Name: new_id, Length: 787, dtype: int64


Each input has 74 QC test, and output products at least 10 inputs. It is impossible to work with this many features in the model. with lack of information I will eliminate some features. But in real life scenerio, I leverage my domain knowledge to identify the key input products that have a significant impact on the output. I focus on those products that are known to have a strong influence on the manufacturing process or are likely to affect the yield.

4- Merge Dataframes

In [71]:
#Merge YIELD_VALUE on new_id and date (Batch_Genealogy df and Combined_Yields_full df)

merged_df= pd.merge(Batch_Genealogy, Combined_Yields_full,on=["MANUFACTURE_DATE","new_id"])



In [72]:
merged_df

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,NEW_MATERIAL_DESCRIPTION,new_id,output_id,YIELD_VALUE
0,R104490026,2022-02-03,232.10,KG,101,hep_so,122812P10449,0,1.092185
1,R104490027,2022-02-11,253.55,KG,101,hep_so,122813P10449,0,1.065421
2,R104490036,2022-09-16,265.10,KG,101,hep_so,122825P10449,0,1.094054
3,R104490023,2021-11-05,223.90,KG,101,hep_so,121843P10449,0,1.082375
4,R104500007,2021-10-21,227.20,KG,101,hep_so,121823P10450,0,1.112719
...,...,...,...,...,...,...,...,...,...
91,R104490011,2021-03-04,233.00,KG,101,hep_so,121827P10449,0,1.088523
92,R104490037,2022-10-06,225.70,KG,101,hep_so,122826P10449,0,1.124211
93,R104490032,2022-06-23,270.60,KG,101,hep_so,122821P10449,0,1.094243
94,R104490031,2022-06-16,236.90,KG,101,hep_so,122819P10449,0,1.063160


In [73]:
unique_combinations = merged_df[['PROCESS_ORDER_NUMBER', 'UNIT_OF_MEASURE','MOVEMENT_TYPE','MANUFACTURE_DATE','NEW_MATERIAL_DESCRIPTION','new_id','output_id']].nunique()
print("# of unique: ")
print(unique_combinations)

# of unique: 
PROCESS_ORDER_NUMBER        96
UNIT_OF_MEASURE              1
MOVEMENT_TYPE                1
MANUFACTURE_DATE            94
NEW_MATERIAL_DESCRIPTION     1
new_id                      96
output_id                    1
dtype: int64


5- Standardization

6- Feature Regularization

7- Save the Data